In [157]:
using QuantumLattices
using FileIO
# using SymPy: symbols
using ExactDiagonalization
using LinearAlgebra

function count1(x)
    sum = 0
    while x != 0
        sum += x % 2
        x = x ÷ 2
    end
    sum
end

function addChemicalP!(mu, m)
    for i in 1:2^(2*N)
        m[i, i] += mu * count1(i-1)
    end
end

addChemicalP! (generic function with 1 method)

In [159]:
N = 6

unitcell = Lattice(:Ring,
    [Point(PID(1), [0.0])],
    vectors=[[1.0]],
    neighbors=1
    )

lattice = Lattice(unitcell, translations"6P")
hilbert = Hilbert(pid=>Fock{:f}(norbital=1, nspin=2) for pid in lattice.pids)

t = Hopping(:t0, -1.0, 1)
U = Hubbard(:U, 4.0)

Hubbard{4}(id=U, value=4.0, bondkind=0, factor=1.0)

In [160]:
bases = BinaryBases(1:2*N)
ed = ED(lattice, hilbert, (U, t), TargetSpace(bases))
a = matrix(ed).matrix
addChemicalP!(-2.0, a)

In [161]:
ed_s2 = ED(lattice, hilbert, (Hubbard(:U, -2.0), Hopping(:t0, -0.0, 1)), TargetSpace(bases))
O = matrix(ed_s2).matrix
addChemicalP!(+1.0, O)

In [153]:
betas = [10.0 / x for x in 1:40]
result = []

for beta in betas
    x1 = tr(exp(Matrix(-beta*a)))
    x2 = tr(Matrix(O) * exp(Matrix(-beta*a)))
    println(1/beta, " => ", (0.75 / N) * (x2 / x1))
    append!(result, (0.75 / N) * (x2 / x1))
end

0.1 => 0.5834704016960006
0.2 => 0.5866594414669681
0.3 => 0.5935701458638133
0.4 => 0.5987248933531623
0.5 => 0.6008174714426087
0.6 => 0.6004120076653501
0.7 => 0.5981376162780363
0.8 => 0.5944928499808774
0.8999999999999999 => 0.5898678268565686
1.0 => 0.5845691800072841
1.1 => 0.578837136793515
1.2 => 0.5728579602371566
1.2999999999999998 => 0.5667738904766005
1.4 => 0.5606915174319858
1.5 => 0.5546889233298149
1.6 => 0.5488217203022637
1.7 => 0.5431280655880201
1.7999999999999998 => 0.537632749711397
1.9000000000000001 => 0.5323504714469417
2.0 => 0.5272884217321018
2.1 => 0.5224482958754713
2.2 => 0.5178278428989377
2.3000000000000003 => 0.5134220464917422
2.4 => 0.5092240167168419
2.5 => 0.5052256570525853
2.5999999999999996 => 0.5014181584608568
2.7 => 0.49779236125556026
2.8 => 0.49433901658025337
2.9 => 0.4910489720989252
3.0 => 0.48791330080555406
3.1 => 0.48492338740190916
3.2 => 0.48207098323901754
3.3 => 0.47934823815725697
3.4 => 0.47674771552100115
3.5 => 0.474262395187